In [1]:
import wave
import numpy as np
import python_speech_features as ps
import os
import glob
import pickle

In [2]:
# Defs

epsilon = 1e-5

def wgn(x, snr):
    snr = 10**(snr/10.0)
    xpower = np.sum(x**2)/len(x)
    npower = xpower / snr
    return np.random.randn(len(x)) * np.sqrt(npower)

def read_file(filename):
    file = wave.open(filename,'r')    
    params = file.getparams()
    nchannels, sampwidth, framerate, wav_length = params[:4]
    str_data = file.readframes(wav_length)
    wavedata = np.fromstring(str_data, dtype = np.short)
    # librosa.load(wav_file_path + orig_wav_file, sr=sr)
    time = np.arange(0,wav_length) * (1.0/framerate)
    file.close()
    return wavedata, time, framerate

def generate_label(control):
    label = -1
    if(control == 'cc'):
        label = 0
    elif(control == 'cd'):
        label = 1
    return label

# def load_data():
#     f = open('./zscore40.pkl','rb')
#     mean1,std1,mean2,std2,mean3,std3 = pickle.load(f)
#     return mean1,std1,mean2,std2,mean3,std3

In [3]:
# cd: 1358
# cc: 1476

epsilon = 1e-5
# tnum = 159 #the number of test utterance
# vnum = 198
# test_num = 220 #the number of test 2s segments
# valid_num = 236
# train_num = 1358 + 1476
filter_num = 40

# pernums_test = np.arange(tnum)#remerber each utterance contain how many segments
# pernums_valid = np.arange(vnum)
rootdir = 'train/Normalised_audio-chunks'

In [4]:
mean1,std1,mean2,std2,mean3,std3 = 0,1,0,1,0,1
ccnum = 1358 #0
cdnum = 1476 #1
pernum = 300

In [5]:
# train_label = np.empty((train_num,1), dtype = np.int8)
# test_label = np.empty((test_num,1), dtype = np.int8)
# valid_label = np.empty((valid_num,1), dtype = np.int8)

# train_data = np.empty((train_num,300,filter_num,3),dtype = np.float32)
# test_data = np.empty((test_num,300,filter_num,3),dtype = np.float32)
# valid_data = np.empty((valid_num,300,filter_num,3),dtype = np.float32)

In [6]:
# count = 0
# for control in os.listdir(rootdir):
    
#     sub_dir = rootdir + '/' + control
    
#     for sample in os.listdir(sub_dir):
#         data, time, rate = read_file(sub_dir+'/'+sample)
# #         mel_spec = ps.logfbank(data,rate,nfilt = filter_num)
#         time = time.shape[0] 
#         if(time <= 300):
#             count += 1
#         else:
#             count += 2
# print(count)

count = 2974
train_label = np.empty((2974,2), dtype = np.int8)
train_data = np.empty((2974,300,filter_num,3),dtype = np.float32)
pernums = np.arange(2974)

In [7]:
tnum = 0
vnum = 0
train_num = 0
test_num = 0
valid_num = 0
train_control = {'cc':0,'cd':0}
test_control = {'cc':0,'cd':0}
valid_control = {'cc':0,'cd':0}

In [8]:
for control in os.listdir(rootdir):
    
    sub_dir = rootdir + '/' + control
    
    for sample in os.listdir(sub_dir):
        data, time, rate = read_file(sub_dir+'/'+sample)
        mel_spec = ps.logfbank(data,rate,nfilt = filter_num)
        delta1 = ps.delta(mel_spec, 2)
        delta2 = ps.delta(delta1, 2)
#             print(time)
        time = mel_spec.shape[0] 

        if(time <= 300):
            pernums[train_num] = 1
            part = mel_spec
            delta11 = delta1
            delta21 = delta2
            part = np.pad(part,((0,300 - part.shape[0]),(0,0)),'constant',constant_values = 0)
            delta11 = np.pad(delta11,((0,300 - delta11.shape[0]),(0,0)),'constant',constant_values = 0)
            delta21 = np.pad(delta21,((0,300 - delta21.shape[0]),(0,0)),'constant',constant_values = 0)
            train_data[train_num,:,:,0] = (part -mean1)/(std1+epsilon)
            train_data[train_num,:,:,1] = (delta11 - mean2)/(std2+epsilon)
            train_data[train_num,:,:,2] = (delta21 - mean3)/(std3+epsilon)

#                 em = generate_label(control)
            train_label[train_num][0] = generate_label(control)
            train_label[train_num][1] = 1
            train_control[control] = train_control[control] + 1
            train_num = train_num + 1
        else:
            pernums[train_num] = 2
            for i in range(2):
                if(i == 0):
                    begin = 0
                    end = begin + 300
                else:
                    begin = time - 300
                    end = time

                part = mel_spec[begin:end,:]
                delta11 = delta1[begin:end,:]
                delta21 = delta2[begin:end,:]
                train_data[train_num,:,:,0] = (part -mean1)/(std1+epsilon)
                train_data[train_num,:,:,1] = (delta11 - mean2)/(std2+epsilon)
                train_data[train_num,:,:,2] = (delta21 - mean3)/(std3+epsilon)

                train_label[train_num][0] = generate_label(control)
                train_label[train_num][1] = 2
                train_num = train_num + 1
                


C:\Users\Asus\AppData\Local\Temp/ipykernel_18204/4235171515.py:16: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  wavedata = np.fromstring(str_data, dtype = np.short)


In [10]:
train_label[569]

array([0, 2], dtype=int8)

In [ ]:
train_control

In [ ]:
np.shape(train_label)

In [11]:
train_label[2000]

array([1, 1], dtype=int8)

In [9]:
from sklearn.model_selection import train_test_split
train_size=0.8

X_train, X_rem, y_train, y_rem = train_test_split(train_data,train_label, train_size=0.8)

In [10]:
np.shape(X_train)

(2379, 300, 40, 3)

In [11]:
test_size = 0.5
X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem, test_size=0.5)

In [12]:
print(X_train.shape), print(y_train.shape)
print(X_valid.shape), print(y_valid.shape)
print(X_test.shape), print(y_test.shape)

(2379, 300, 40, 3)
(2379, 2)
(297, 300, 40, 3)
(297, 2)
(298, 300, 40, 3)
(298, 2)


(None, None)

In [13]:
y_labels = np.empty((2379,1), dtype = np.int8)
y_pernums = np.empty((2379,1), dtype = np.int8)

for count in range(2379):
    y_labels[count] = y_train[count][0]


In [14]:
valid_labels = np.empty((297,1), dtype = np.int8)
valid_pernums = np.empty((297,1), dtype = np.int8)

for count in range(297):
    valid_labels[count] = y_valid[count][0]
    valid_pernums[count] = y_valid[count][1]

In [15]:
y_valid

array([[1, 1],
       [1, 1],
       [0, 2],
       [1, 1],
       [0, 1],
       [1, 1],
       [1, 1],
       [0, 2],
       [0, 2],
       [0, 1],
       [0, 1],
       [0, 1],
       [1, 1],
       [0, 1],
       [1, 1],
       [1, 1],
       [1, 1],
       [0, 1],
       [1, 1],
       [0, 1],
       [1, 1],
       [0, 1],
       [0, 1],
       [1, 1],
       [1, 1],
       [1, 1],
       [1, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [1, 2],
       [1, 1],
       [1, 1],
       [0, 1],
       [0, 2],
       [1, 1],
       [1, 1],
       [1, 1],
       [0, 1],
       [1, 1],
       [1, 1],
       [0, 1],
       [0, 1],
       [1, 1],
       [1, 1],
       [1, 1],
       [1, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [1, 1],
       [0, 1],
       [1, 1],
       [1, 1],
       [0, 1],
       [1, 1],
       [1, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [1, 1],
       [1, 1],
       [1,

In [16]:
test_labels = np.empty((298,1), dtype = np.int8)
test_pernums = np.empty((298,1), dtype = np.int8)

for count in range(298):
    test_labels[count] = y_test[count][0]
    test_pernums[count] = y_test[count][1]

In [20]:
y_train

array([[1, 1],
       [1, 1],
       [1, 2],
       ...,
       [0, 1],
       [1, 1],
       [1, 1]], dtype=int8)

In [17]:
output = './adress.pkl'
f=open(output,'wb') 
pickle.dump((X_train,y_labels,X_test,test_labels,X_valid,valid_labels,valid_pernums),f)
f.close() 